In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset('docred', trust_remote_code=True)
annotated = pd.DataFrame(dataset['train_annotated'])
distant = pd.DataFrame(dataset['train_distant'])

Data Format:
(source: https://github.com/thunlp/DocRED/blob/master/data/README.md)
{
  'title',
  'sents':     [
                  [word in sent 0],
                  [word in sent 1]
               ]
  'vertexSet': [
                  [
                    { 'name': mention_name, 
                      'sent_id': mention in which sentence, 
                      'pos': postion of mention in a sentence, 
                      'type': NER_type}
                    {anthor mention}
                  ], 
                  [anthoer entity]
                ]
  'labels':   [
                {
                  'h': idx of head entity in vertexSet,
                  't': idx of tail entity in vertexSet,
                  'r': relation,
                  'evidence': evidence sentences' id
                }
              ]
}

In [60]:
# Sentences
from _RE import join_text

sents = annotated.iloc[3]['sents']

for sent in sents:
    print(' '.join(sent))

print('JOINED:')
print(join_text(sents))

Pacific Fair is a major shopping centre in Broadbeach Waters on the Gold Coast , Queensland , Australia .
It was Queensland 's largest regional shopping centre until 2006 .
Pacific Fair was developed by Hooker Retail Developments and opened in 1977 on what was swampland with 96 specialty stores and two anchor tenants .
Since then , Pacific Fair has undergone numerous expansions and has grown to have more than 300 specialty stores and four anchor tenants .
In January 2014 , work began on a major redevelopment project to meet the predicted regional growth on the Gold Coast .
Prior to the redevelopment , the shopping centre had four main major stores including a four - level Myer , Kmart , Target , Coles and Toys ' R ' Us .
Daimaru operated in the centre before its Australian withdrawal , albeit briefly .
It also had a 12-screen Birch Carroll and Coyle Cinema ( re - opened as Event Cinemas in late 2015 ) .
Pacific Fair is a major public transport interchange on the Gold Coast , serviced b

In [9]:
labels = annotated.iloc[0]['labels']
for l in labels:
    print(l, labels[l])

head [0, 0, 12, 2, 2, 4, 5, 3, 3, 3, 1, 1, 10]
tail [2, 4, 4, 4, 3, 3, 4, 2, 4, 4, 2, 4, 4]
relation_id ['P159', 'P17', 'P17', 'P17', 'P131', 'P150', 'P17', 'P150', 'P131', 'P17', 'P131', 'P17', 'P17']
relation_text ['headquarters location', 'country', 'country', 'country', 'located in the administrative territorial entity', 'contains administrative territorial entity', 'country', 'contains administrative territorial entity', 'located in the administrative territorial entity', 'country', 'located in the administrative territorial entity', 'country', 'country']
evidence [[0], [2, 4, 7], [6, 7], [0], [0], [0], [0, 3], [0], [0, 3], [0, 3], [0, 3], [0, 3], [4]]
